*This tutorial is created by [Ahmadreza Attarpour](https://github.com/AAttarpour), and [Jonas Osman](https://github.com/jono3030).*

*Unpublished results.*

#**Large tera-voxel size of whole-brain light sheet microscopy images**

- Tissue clearing and light sheet fluorescence microscopy (LSFM) produce exceedingly large (tera-voxel scale), complex, multichannel three-dimensional (3D) datasets
- E.g. a complete snapshot of neural activity brain-wide reflected in levels of c-Fos (~1 TB of 3D data per brain sample)

![](https://drive.google.com/uc?export=view&id=1DJmgDfK-GR52fljiw1XQBCOv5o5vdU5i)

#**Methodological Gap**

# How do we map ***local / subregional / laminar specific*** activity across the brain using tera-voxel size LSFM data.

# **What is ACE?**

*AI-based Cartography of Ensembles* (ACE) is an ***end-to-end*** pipeline to map local and laminar neuronal activity leveargin deep learning, advanced cluster wise analysis, and whole-brain tera-voxel size light sheet flurorescence microscopy (LSFM) dataset.

![](https://drive.google.com/uc?export=view&id=1RVfvArucG9-cgvEVhsEiOm9vIO60hddM)

#**ACE Highlights**

- Cutting-edge ***vision transformer*** and ***CNN***-based deep learning architectures trained on very large LSFM datasets (link to sample data and refer to example section) to map brain-wide neuronal activity.
- Optimized ***cluster-wise statistical analysis*** with a ***threshold-free enhancement*** approach to chart subpopulation-specific effects at the laminar and local level, ***without restricting the analysis to atlas-defined regions***.
- Modules for providing DL model uncertainty estimates and fine-tuning.
- Interface with MIRACL registration to create study-specific atlases.
- Ability to account for covariates at the cluster level and map the connectivity between clusters of activations.

#**ACE API**

- We included ACE pipeline in our open source [**MIRACL software**](https://miracl.readthedocs.io/en/latest/).
- We provide a build script to automatically create either a [**Docker**](https://miracl.readthedocs.io/en/latest/installation/installation.html) or [**Singularity**](https://miracl.readthedocs.io/en/latest/installation/installation.html) image.
- useful links:
  1. [MIRACL Github page](https://miracl.readthedocs.io/en/latest/index.html)
  2. [MIRACL readdoc page](https://github.com/AICONSlab/MIRACL)
  3. [ACE tutorial](https://miracl.readthedocs.io/en/latest/tutorials/workflows/ace_flow/ace_flow.html)

#**ACE CLI**

To get more information about the workflow and its required arguments use the following command on the cli:



```
$ miracl flow ace -h
```



![](https://drive.google.com/uc?export=view&id=1nzlNhUU0STBz6getlhyYc0y9poT5yaTV)

#**Some more useful information can be found by using:**


```
$ miracl flow ace -hv
```

![](https://drive.google.com/uc?export=view&id=16YAn3I5c1NL7TC8Hhy2UA979AqKQO53I)

ACE provides an estimate of model uncertainty/confidence. With uncertainty in hands, we can truly evaluate model performance and increase its performance using either post processing or fine tune the model (fine tuning scripts are already incorporated in ACE)

#**Example of running ACE on single subject (segmenation + registration + voxelization + warping) using Vision Transformer (UNETR) model with MC dropout:**



```
$ miracl flow ace \
  -s ./hemisphere_data_subject1/ \
  -sao ./output_dir \
  -sam ensemble \
  -sar 1.8 1.8 2 \
  -samc
```




- ```./hemisphere_data_subject1/``` directory contains Z slices of 3D raw LSFM data from one hemisphere of a mouse brain. The voxel size of the data is 1.8x1.8x2 um.

- by selecting ```ensemble``` and ```samc``` we will use ensemble of ACE UNETR and UNET models; each with MC dropout (ensemble of ensembles).



#**Main Outputs**

ACE creates the following directories for each subject:



* Registration outputs:

```
- conv_final # conversion (tiff to nifti) output
- reg_final  # main registration output
- clar_allen_reg # registration output / pre-liminary files
- reg_final  # main registration output
```
* Segmentation outputs:

```
- seg_final # segmentation output including model(s) outputs and uncertainty estimates
- vox_final # segmentation maps are voxelized using a conv filter to avoid data loss in warping and downsampling
- warp_final # voxelized maps are warped to study specific atlas
```



# Here are the reults of ACE on one single subject:

![](https://drive.google.com/uc?export=view&id=1ZMSggGeEbZjIcx-jZGZuAvFAQ5U8QT96)

# Here are the reults of ACE registration:
![](https://drive.google.com/uc?export=view&id=1t6AQQ5gE3NYBrSDf8VhmRHZzv1k6_KDs)





#**Example of running ACE flow on two groups of study and multiple subjects**



```
$ miracl flow ace \
   -c ./non_walking \
   -e ./walking \
   -sao ./output_dir \
   -sar 1.4 1.4 5 \
   -sam ensemble \
   -samc \
   -rcao ALI \
   -rcav 10

```

**Arguments:**

- Each ```./non_walking & ./walking``` directories contains 3 subjects. Each should have a directory corresponding the Z slices of a 3D whole-brain LSFM data with the resolution of 1.4x1.4.5 um.

**Segmnetation:**

- by selecting ```ensemble``` and ```samc``` we will use ensemble of ACE UNETR and UNET models; each with MC dropout (ensemble of ensembles).

**Registration**

- by selecting ```rcao ALI``` and ```rcav 10```, ACE will use 10x for downsampling of the input image to registration and ```AIR``` code for orienting it to a study specific atlas (default is Allen atlas)

**Cluster wise analysis**

- for this example, ACE will use all default parameters for its cluster wise threshold free enhancement (TFCE) statistical test such as TFCE step (5) and start (0.01), and number of permutations (500)



#**Main Outputs**

In addition to the directories ACE creates for one subject input, the following directories also will be generated:

* Cluster wise analysis outputs:

```
heatmap_final # cell density heatmaps, demonstrating activity hotspots between two groups of study.
cluster_final # cluster-wise analysis output including p_value and f_stats maps.
```

*Specifically, in addition to cluster wise p_value and f_stats, ACE provides an excel file, summarizing information about each signficant cluster found. These information consists of cluster volume, effect size, and the regions of the brain (with percentage) each cluster spanned out.*

# Here are the reults of ACE on two groups and multiple subjects:

# first, let's look at one subject's segmentation + registration + warping:
![](https://drive.google.com/uc?export=view&id=1sdtjvn3SOMR9gnx02RN7fLsx7I7kNzli)
Data rendering was done using `Napari`, `Freeview`, and `Brainrender`

# here are the cluster-wise analysis:
![](https://drive.google.com/uc?export=view&id=18dQlLXUMjgJq2Ixl0xE5bnwdJWdGH8lt)



ACE optimized cluster-wise statistical analysis with a threshold-free enhancement approach charts subpopulation-specific effects at the laminar and local level, without restricting the analysis to atlas-defined regions.
E. g. here are the top (sorted by effect size) signficant clusters of activation in the above example from the `.csv` file (ACE's output):

![](https://drive.google.com/uc?export=view&id=10BukOCrOelOWPXYOkRYEz9UDXA3JcZxA)


#**Summary**

- ACE is an end-to-end pipeline to map local/laminar activity at the level of whole brain.
- ACE segmentation module is based cutting-edge deep learning models (both vision transformer and CNN) where trained LSFM data from multiple centers and evaluated on unseen datasets (manuscript under review).
- Provides cluster-wise statistical analysis with high specificity & sensitivity in an atlas-agnostic manner.
- Due to modular nature of the pipeline, ACE is easily extendable to map other cell types in 3D microscopy data.   
